In [ ]:
## w Windows 10/11 z zainstalowanym Python'em
## kod nalezy wykonać w środowisku wirtualnym python (venv)
## w katakogu roboczym nalezy utworzyć plik .env w którym zapisumy OPENAI_API_KEY. lodad_dotenv odczyta klucz.

## dodatkowe biblioteki do zainstalowania w venv
# pip install langchain openai 
# pip install unstructured

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

from pprint import pprint

In [ ]:
# wczytujemy wszystkie pliki *.txt z folderu

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

text_loader_kwargs={'autodetect_encoding': True}

loader = DirectoryLoader('../../sciezka_do_tekstow_do_wczytania/', glob="**/*.txt", show_progress=True, use_multithreading=True, loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)  #należy wpisać właściwą ścieżkę do katalogu z tekstami *.txt
docs = loader.load()

In [ ]:
# dzielimy pliki na mniejsze częsci

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=50)
all_splits = splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
#do wykonania tylko raz. Na dysku zostanie zapisana baza danych, do której dostęp uzyskujemy w następnym kroku

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings(), persist_directory="./texts_chroma_db")

In [ ]:
# ładujemy baze wektorową z folderu

vectorstore = Chroma(persist_directory="./texts_chroma_db", embedding_function=OpenAIEmbeddings())

In [ ]:
#test - miejsce na Twoje pytanie

query = "Miejsce na twoje pytanie?"
docs = vectorstore.similarity_search(query)

print(docs[1].page_content)

In [ ]:
## teraz chatbot
# parametry typu model i temparure do zmiany

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [ ]:
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
answer = qa("miejsce na twoje pytanie?")

In [ ]:
print(answer["answer"])